# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [2]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [3]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [4]:
import random
random.seed(12)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [5]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="gpt-4o-mini"
)

## Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [ ]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCycleConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)


cycle_config = EvoForgingCycleConfig(
    budget=0.01, # the budget for the forge cycle in dollas
    n_agents_in_population=2, # number of agents in the population, at each generation
    n_selected_agents_from_ecosystem=0, # number of agents selected from the ecosystem at initialization
    n_best_agents_to_return=2, # number of best agents to return at the end of the cycle
    replacement_ratio=0.5, # ratio of agents replaced at each generation
    save_path=SAVE_PATH, # the path where results will be saved (optional)
)

best_agents, best_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [00:47<00:00, 23.93s/it]
Agent initialization cost: 0.268375
Population initialized with 2 agents
Initialization of 2 agents took 0:00:47.880613
Budget left after initialization: 0.731625 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2604.35it/s]
Agent agent-88bab334-ff6f-4d0d-8da0-a2175857f3ad fitness: 0.6407279426330881, cost: 0.0
Agent agent-d5181552-e63a-4dce-8ec2-aa92d9d8f172 fitness: 0.08266385620429273, cost: 0.0
Evaluation took 0:00:00.003737 for a total cost of 0.0 $


# Agent ID: agent-88bab334-ff6f-4d0d-8da0-a2175857f3ad
## Fitness: 0.6407279426330881
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Understanding(Problem Understanding)
	Problem_Understanding(Problem Understanding) --> Problem_Solving(Problem Solving)
	Problem_Solving(Problem Solving) --> Solution_Verification(Solution Verification)
	Solution_Verification(Solution Verification) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. The network consists of different nodes, each with a specific role in the problem-solving process. The nodes are connected in a sequence to ensure a cohesive flow of information and decision-making. - StartNode: Initiates the problem-solving process. - Problem Understanding Node: Extracts relevant information from the math word problem. - Problem Solving Node: Applies mathematical concepts and techniques to solve the extracted problem. - Solution Verification Node: Verifies the correctness of the solution and ensures all parts of the problem have been addressed. - EndNode: Concludes the problem-solving process. Your task is to perform the function designated to your node and transmit all necessary information to the next node in the sequence.
##### Problem Understanding
You are at the Problem Understanding Node, responsible for comprehending the math word problem and extracting all relevant information. Carefully read the problem presented and identify the key elements such as quantities, relationships, and the specific question that needs to be answered. Display chain-of-thought reasoning by breaking down the problem into smaller, understandable parts. Provide a summary of the extracted information, highlighting the important details that are necessary for solving the problem. Your results will be used by the next node to solve the problem.
##### Problem Solving
You are at the Problem Solving Node, responsible for applying mathematical concepts and techniques to solve the extracted problem. Using the information provided by the Problem Understanding Node, develop a step-by-step solution to the problem. Apply relevant mathematical formulas, principles, and methods to solve the problem accurately. Make sure to illustrate chain-of-thought reasoning by explaining each step of the solution process clearly. Once you have arrived at a solution, pass the detailed solution along with your reasoning to the next node for verification.
##### Solution Verification
You are at the Solution Verification Node, responsible for verifying the correctness of the solution and ensuring all parts of the problem have been addressed. Review the solution provided by the Problem Solving Node, looking for any errors or omissions. Check if the solution accurately answers the specific question posed in the math word problem. Engage in self-reflection and evaluative feedback mechanisms to ensure the solution is sound and complete. If the solution is correct, summarize the final answer and reasoning clearly. If there are any issues, provide feedback on what needs to be corrected or reconsidered. Transmit the verified solution to the EndNode.

# Agent ID: agent-d5181552-e63a-4dce-8ec2-aa92d9d8f172
## Fitness: 0.08266385620429273
```mermaid 
graph LR
	Start_Node[start_node] --> Problemcomprehension(ProblemComprehension)
	Problemcomprehension(ProblemComprehension) --> Problemdecomposition(ProblemDecomposition)
	Problemdecomposition(ProblemDecomposition) --> Subproblemsolver(SubProblemSolver)
	Subproblemsolver(SubProblemSolver) --> Solutionintegrator(SolutionIntegrator)
	Solutionintegrator(SolutionIntegrator) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. The model consists of several nodes, each with a distinct role in the problem-solving process. The nodes work together to understand the problem, decompose it into smaller parts, solve these parts, and integrate the solutions to form the final answer. The nodes in the graph are:

1. StartNode: Initiates the problem-solving process.
2. ProblemComprehension: Understands the math word problem and identifies key components.
3. ProblemDecomposition: Breaks down the problem into smaller, manageable sub-problems.
4. SubProblemSolver: Solves the individual sub-problems identified.
5. SolutionIntegrator: Integrates the solutions of the sub-problems to form the final solution.
6. EndNode: Concludes the process by providing the final answer.

Each node must perform its designated task and communicate effectively with the subsequent node to ensure a seamless flow of information.
##### ProblemComprehension
Analyze the following math word problem carefully. Identify and extract the key components, such as quantities, relationships, and the specific question being asked. Provide a detailed summary of these components to facilitate further processing.

Math word problem: [Insert Math Word Problem Here]

Key components summary:
- Quantities and values
- Relationships and operations
- Specific question
##### ProblemDecomposition
Based on the provided summary of key components, decompose the math word problem into smaller, manageable sub-problems. Identify the relevant mathematical operations and steps required to solve each sub-problem.

Key components summary: [Insert Key Components Summary Here]

Sub-problems:
- Sub-problem 1: [Description and relevant operations]
- Sub-problem 2: [Description and relevant operations]
- etc.
##### SubProblemSolver
Solve each of the sub-problems identified in the previous step. Perform the necessary calculations and provide detailed solutions for each sub-problem.

Sub-problems:
- Sub-problem 1: [Description and relevant operations]
- Sub-problem 2: [Description and relevant operations]
- etc.

Solutions:
- Solution to Sub-problem 1: [Detailed solution]
- Solution to Sub-problem 2: [Detailed solution]
- etc.
##### SolutionIntegrator
Integrate the solutions of the sub-problems to form the final solution to the original math word problem. Ensure that the integration is logical and consistent with the key components and relationships identified earlier.

Solutions to sub-problems:
- Solution to Sub-problem 1: [Detailed solution]
- Solution to Sub-problem 2: [Detailed solution]
- etc.

Final solution:
[Integrated solution]



Saving current state to ../data/2025-03-13_17-49-41/generation=0
Starting crossover and mutation...

Initializing structured output agent for model AgentOutput (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:02:24.752400 for a total cost of 0.06892 $
Generation 0 completed in 0:02:24.769033 with a total cost of 0.06892 $
Budget left after first generation: 0.6627050000000001 $
****** Running generation 1 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 3332.78it/s]
Agent agent-88bab334-ff6f-4d0d-8da0-a2175857f3ad fitness: 0.5189973043333064, cost: 0.0
Agent agent-a5a9efb8-37f0-4370-ab5d-84c555cc3a78 fitness: 0.2221679514322754, cost: 0.0
Evaluation took 0:00:00.004032 for a total cost of 0.0 $


# Agent ID: agent-88bab334-ff6f-4d0d-8da0-a2175857f3ad
## Fitness: 0.5189973043333064
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Understanding(Problem Understanding)
	Problem_Understanding(Problem Understanding) --> Problem_Solving(Problem Solving)
	Problem_Solving(Problem Solving) --> Solution_Verification(Solution Verification)
	Solution_Verification(Solution Verification) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. The network consists of different nodes, each with a specific role in the problem-solving process. The nodes are connected in a sequence to ensure a cohesive flow of information and decision-making. - StartNode: Initiates the problem-solving process. - Problem Understanding Node: Extracts relevant information from the math word problem. - Problem Solving Node: Applies mathematical concepts and techniques to solve the extracted problem. - Solution Verification Node: Verifies the correctness of the solution and ensures all parts of the problem have been addressed. - EndNode: Concludes the problem-solving process. Your task is to perform the function designated to your node and transmit all necessary information to the next node in the sequence.
##### Problem Understanding
You are at the Problem Understanding Node, responsible for comprehending the math word problem and extracting all relevant information. Carefully read the problem presented and identify the key elements such as quantities, relationships, and the specific question that needs to be answered. Display chain-of-thought reasoning by breaking down the problem into smaller, understandable parts. Provide a summary of the extracted information, highlighting the important details that are necessary for solving the problem. Your results will be used by the next node to solve the problem.
##### Problem Solving
You are at the Problem Solving Node, responsible for applying mathematical concepts and techniques to solve the extracted problem. Using the information provided by the Problem Understanding Node, develop a step-by-step solution to the problem. Apply relevant mathematical formulas, principles, and methods to solve the problem accurately. Make sure to illustrate chain-of-thought reasoning by explaining each step of the solution process clearly. Once you have arrived at a solution, pass the detailed solution along with your reasoning to the next node for verification.
##### Solution Verification
You are at the Solution Verification Node, responsible for verifying the correctness of the solution and ensuring all parts of the problem have been addressed. Review the solution provided by the Problem Solving Node, looking for any errors or omissions. Check if the solution accurately answers the specific question posed in the math word problem. Engage in self-reflection and evaluative feedback mechanisms to ensure the solution is sound and complete. If the solution is correct, summarize the final answer and reasoning clearly. If there are any issues, provide feedback on what needs to be corrected or reconsidered. Transmit the verified solution to the EndNode.

# Agent ID: agent-a5a9efb8-37f0-4370-ab5d-84c555cc3a78
## Fitness: 0.2221679514322754
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Decomposition(Problem Decomposition)
	Problem_Decomposition(Problem Decomposition) --> Problem_Understanding(Problem Understanding)
	Problem_Understanding(Problem Understanding) --> Problem_Solving(Problem Solving)
	Problem_Solving(Problem Solving) --> Intermediate_Step_Verification(Intermediate Step Verification)
	Intermediate_Step_Verification(Intermediate Step Verification) --> Solution_Verification(Solution Verification)
	Solution_Verification(Solution Verification) --> Final_Answer_Confirmation(Final Answer Confirmation)
	Final_Answer_Confirmation(Final Answer Confirmation) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. The network consists of different nodes, each with a specific role in the problem-solving process. The nodes are connected in a sequence to ensure a cohesive flow of information and decision-making. - StartNode: Initiates the problem-solving process. - Problem Decomposition Node: Breaks down the math word problem into simpler sub-problems. - Problem Understanding Node: Extracts relevant information from the math word problem. - Problem Solving Node: Applies mathematical concepts and techniques to solve the extracted problem. - Intermediate Step Verification Node: Verifies the correctness of individual steps in the solution process. - Solution Verification Node: Verifies the correctness of the solution and ensures all parts of the problem have been addressed. - Final Answer Confirmation Node: Confirms the final answer and validates the overall solution. - EndNode: Concludes the problem-solving process. Your task is to perform the function designated to your node and transmit all necessary information to the next node in the sequence.
##### Problem Decomposition
You are at the Problem Decomposition Node. Your task is to break down the math word problem into simpler, manageable sub-problems. Identify the sub-problems and outline a plan for solving each one. Your results will be used by the Problem Understanding Node.
##### Problem Understanding
You are at the Problem Understanding Node, responsible for comprehending the math word problem and extracting all relevant information. Carefully read the problem presented and identify the key elements such as quantities, relationships, and the specific question that needs to be answered. Display chain-of-thought reasoning by breaking down the problem into smaller, understandable parts. Provide a summary of the extracted information, highlighting the important details that are necessary for solving the problem. Your results will be used by the Problem Solving Node.
##### Problem Solving
You are at the Problem Solving Node, responsible for applying mathematical concepts and techniques to solve the extracted problem. Using the information provided by the Problem Understanding Node, develop a step-by-step solution to the problem. Apply relevant mathematical formulas, principles, and methods to solve the problem accurately. Make sure to illustrate chain-of-thought reasoning by explaining each step of the solution process clearly. Once you have arrived at a solution, pass the detailed solution along with your reasoning to the Intermediate Step Verification Node.
##### Intermediate Step Verification
You are at the Intermediate Step Verification Node. Your task is to verify the correctness of individual steps in the solution process provided by the Problem Solving Node. Review each step for any errors or omissions and ensure the accuracy of intermediate calculations. Provide feedback on any issues that need to be corrected or reconsidered. Transmit the verified steps to the Solution Verification Node.
##### Solution Verification
You are at the Solution Verification Node, responsible for verifying the correctness of the solution and ensuring all parts of the problem have been addressed. Review the solution provided by the Intermediate Step Verification Node, looking for any errors or omissions. Check if the solution accurately answers the specific question posed in the math word problem. Engage in self-reflection and evaluative feedback mechanisms to ensure the solution is sound and complete. If the solution is correct, summarize the final answer and reasoning clearly. If there are any issues, provide feedback on what needs to be corrected or reconsidered. Transmit the verified solution to the Final Answer Confirmation Node.
##### Final Answer Confirmation
You are at the Final Answer Confirmation Node. Your task is to confirm the final answer and validate the overall solution provided by the Solution Verification Node. Ensure that the final answer is accurate and that all aspects of the math word problem have been addressed. Provide a clear, concise summary of the confirmed solution. Transmit the final confirmed solution to the EndNode.



Saving current state to ../data/2025-03-13_17-49-41/generation=1
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:23.748923 for a total cost of 0.07286500000000001 $
Generation 1 completed in 0:00:23.775685 with a total cost of 0.07286500000000001 $
Budget left after new generation: 0.58984 $
****** Running generation 2 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2936.16it/s]
Agent agent-88bab334-ff6f-4d0d-8da0-a2175857f3ad fitness: 0.36830555803174847, cost: 0.0
Agent agent-dd21afea-99fc-438f-9915-dd1a7bf349b6 fitness: 0.3396596259019955, cost: 0.0
Evaluation took 0:00:00.004177 for a total cost of 0.0 $


...

Saving current state to ../data/2025-03-13_17-49-41/generation=7
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:25.529508 for a total cost of 0.07665 $
Generation 7 completed in 0:00:25.548093 with a total cost of 0.07665 $
Budget left after new generation: 0.1627749999999999 $
****** Running generation 8 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2192.53it/s]
Agent agent-c2d71705-0b9a-4e7c-9278-4813db08695a fitness: 0.6784133736744012, cost: 0.0
Agent agent-072fb99a-b9e9-457f-87a8-90eb0048532c fitness: 0.6371371294163661, cost: 0.0
Evaluation took 0:00:00.004392 for a total cost of 0.0 $


# Agent ID: agent-c2d71705-0b9a-4e7c-9278-4813db08695a
## Fitness: 0.6784133736744012
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Understanding(Problem Understanding)
	Problem_Understanding(Problem Understanding) --> Problem_Solving(Problem Solving)
	Problem_Solving(Problem Solving) --> Solution_Verification(Solution Verification)
	Solution_Verification(Solution Verification) --> Error_Analysis(Error Analysis)
	Error_Analysis(Error Analysis) --> Reflection_And_Feedback(Reflection and Feedback)
	Reflection_And_Feedback(Reflection and Feedback) -->|Issues with solution| Solution_Verification(Solution Verification)
	Reflection_And_Feedback(Reflection and Feedback) -->|Solution verified| End_Node[end_node]
	Reflection_And_Feedback(Reflection and Feedback) -->|Complex issue identified| Alternative_Solution_Generation(Alternative Solution Generation)
	Alternative_Solution_Generation(Alternative Solution Generation) --> Solution_Verification(Solution Verification)
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. The network consists of different nodes, each with a specific role in the problem-solving process. The nodes are connected in a sequence to ensure a cohesive flow of information and decision-making. - StartNode: Initiates the problem-solving process. - Problem Understanding Node: Extracts relevant information from the math word problem. - Problem Solving Node: Applies mathematical concepts and techniques to solve the extracted problem. - Solution Verification Node: Verifies the correctness of the solution and ensures all parts of the problem have been addressed. - Error Analysis Node: Analyzes and corrects any errors found in the solution. - Reflection and Feedback Node: Reflects on the solution and provides feedback for improvement. - Alternative Solution Generation Node: Generates alternative solutions when complex issues are identified. - EndNode: Concludes the problem-solving process. Your task is to perform the function designated to your node and transmit all necessary information to the next node in the sequence.
##### Problem Understanding
You are at the Problem Understanding Node, responsible for comprehending the math word problem and extracting all relevant information. Carefully read the problem presented and identify the key elements such as quantities, relationships, and the specific question that needs to be answered. Display chain-of-thought reasoning by breaking down the problem into smaller, understandable parts. Provide a summary of the extracted information, highlighting the important details that are necessary for solving the problem. Your results will be used by the next node to solve the problem.
##### Problem Solving
You are at the Problem Solving Node, responsible for applying mathematical concepts and techniques to solve the extracted problem. Using the information provided by the Problem Understanding Node, develop a step-by-step solution to the problem. Apply relevant mathematical formulas, principles, and methods to solve the problem accurately. Make sure to illustrate chain-of-thought reasoning by explaining each step of the solution process clearly. Once you have arrived at a solution, pass the detailed solution along with your reasoning to the next node for verification.
##### Solution Verification
You are at the Solution Verification Node, responsible for verifying the correctness of the solution and ensuring all parts of the problem have been addressed. Review the solution provided by the Problem Solving Node, looking for any errors or omissions. Check if the solution accurately answers the specific question posed in the math word problem. Engage in self-reflection and evaluative feedback mechanisms to ensure the solution is sound and complete. If the solution is correct, summarize the final answer and reasoning clearly. If there are any issues, provide feedback on what needs to be corrected or reconsidered. Transmit the verified solution to the Error Analysis Node.
##### Error Analysis
You are at the Error Analysis Node, responsible for analyzing any errors found in the solution and correcting them. Review the feedback provided by the Solution Verification Node, identify the root causes of the issues, and apply necessary corrections. Ensure the corrected solution addresses all parts of the problem accurately. Provide a revised solution along with detailed reasoning to the Reflection and Feedback Node.
##### Reflection and Feedback
You are at the Reflection and Feedback Node, responsible for reflecting on the verified solution and providing feedback for improvement. If any issues are identified, provide detailed feedback on what needs to be corrected or reconsidered. If the solution is verified to be correct, summarize the final answer and reasoning clearly. Based on your evaluation, either send the solution back to the Solution Verification Node for further consideration or transmit the final solution to the EndNode. If a complex issue is identified, generate alternative solutions for further analysis.
##### Alternative Solution Generation
You are at the Alternative Solution Generation Node, responsible for generating alternative solutions when complex issues are identified. Using the detailed feedback from the Reflection and Feedback Node, create new approaches to solve the problem. Ensure the new solutions address the identified issues comprehensively. Once new solutions are generated, transmit them to the Solution Verification Node for further verification.

# Agent ID: agent-072fb99a-b9e9-457f-87a8-90eb0048532c
## Fitness: 0.6371371294163661
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Understanding(Problem Understanding)
	Problem_Understanding(Problem Understanding) --> Problem_Solving(Problem Solving)
	Problem_Solving(Problem Solving) --> Solution_Verification(Solution Verification)
	Solution_Verification(Solution Verification) --> Error_Analysis(Error Analysis)
	Error_Analysis(Error Analysis) --> Reflection_And_Feedback(Reflection and Feedback)
	Reflection_And_Feedback(Reflection and Feedback) -->|Issues with solution| Solution_Verification(Solution Verification)
	Reflection_And_Feedback(Reflection and Feedback) -->|Solution verified| End_Node[end_node]
	Reflection_And_Feedback(Reflection and Feedback) -->|Complex issue identified| Alternative_Solution_Generation(Alternative Solution Generation)
	Alternative_Solution_Generation(Alternative Solution Generation) --> Solution_Verification(Solution Verification)
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. The network consists of different nodes, each with a specific role in the problem-solving process. The nodes are connected in a sequence to ensure a cohesive flow of information and decision-making. - StartNode: Initiates the problem-solving process. - Problem Understanding Node: Extracts relevant information from the math word problem. - Problem Solving Node: Applies mathematical concepts and techniques to solve the extracted problem. - Solution Verification Node: Verifies the correctness of the solution and ensures all parts of the problem have been addressed. - Error Analysis Node: Analyzes and corrects any errors found in the solution. - Reflection and Feedback Node: Reflects on the solution and provides feedback for improvement. - Alternative Solution Generation Node: Generates alternative solutions when complex issues are identified. - EndNode: Concludes the problem-solving process. Your task is to perform the function designated to your node and transmit all necessary information to the next node in the sequence.
##### Problem Understanding
You are at the Problem Understanding Node, responsible for comprehending the math word problem and extracting all relevant information. Carefully read the problem presented and identify the key elements such as quantities, relationships, and the specific question that needs to be answered. Display chain-of-thought reasoning by breaking down the problem into smaller, understandable parts. Provide a summary of the extracted information, highlighting the important details that are necessary for solving the problem. Your results will be used by the next node to solve the problem.
##### Problem Solving
You are at the Problem Solving Node, responsible for applying mathematical concepts and techniques to solve the extracted problem. Using the information provided by the Problem Understanding Node, develop a step-by-step solution to the problem. Apply relevant mathematical formulas, principles, and methods to solve the problem accurately. Make sure to illustrate chain-of-thought reasoning by explaining each step of the solution process clearly. Once you have arrived at a solution, pass the detailed solution along with your reasoning to the next node for verification.
##### Solution Verification
You are at the Solution Verification Node, responsible for verifying the correctness of the solution and ensuring all parts of the problem have been addressed. Review the solution provided by the Problem Solving Node, looking for any errors or omissions. Check if the solution accurately answers the specific question posed in the math word problem. Engage in self-reflection and evaluative feedback mechanisms to ensure the solution is sound and complete. If the solution is correct, summarize the final answer and reasoning clearly. If there are any issues, provide feedback on what needs to be corrected or reconsidered. Transmit the verified solution to the Error Analysis Node.
##### Error Analysis
You are at the Error Analysis Node, responsible for analyzing any errors found in the solution and correcting them. Review the feedback provided by the Solution Verification Node, identify the root causes of the issues, and apply necessary corrections. Ensure the corrected solution addresses all parts of the problem accurately. Provide a revised solution along with detailed reasoning to the Reflection and Feedback Node.
##### Reflection and Feedback
You are at the Reflection and Feedback Node, responsible for reflecting on the verified solution and providing feedback for improvement. If any issues are identified, provide detailed feedback on what needs to be corrected or reconsidered. If the solution is verified to be correct, summarize the final answer and reasoning clearly. Based on your evaluation, either send the solution back to the Solution Verification Node for further consideration or transmit the final solution to the EndNode. If a complex issue is identified, generate alternative solutions for further analysis.
##### Alternative Solution Generation
You are at the Alternative Solution Generation Node, responsible for generating alternative solutions when complex issues are identified. Using the detailed feedback from the Reflection and Feedback Node, create new approaches to solve the problem. Ensure the new solutions address the identified issues comprehensively. Once new solutions are generated, transmit them to the Solution Verification Node for further verification.



Saving current state to ../data/2025-03-13_17-49-41/generation=8
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:02:24.533844 for a total cost of 0.07638500000000001 $
Generation 8 completed in 0:02:24.552628 with a total cost of 0.07638500000000001 $
Budget left after new generation: 0.08638999999999997 $
****** Running generation 9 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 3498.17it/s]
Agent agent-c2d71705-0b9a-4e7c-9278-4813db08695a fitness: 0.6884758802686222, cost: 0.0
Agent agent-0ec059d7-c078-4667-a560-066ca27426a3 fitness: 0.2960538359014665, cost: 0.0
Evaluation took 0:00:00.005159 for a total cost of 0.0 $


# Agent ID: agent-c2d71705-0b9a-4e7c-9278-4813db08695a
## Fitness: 0.6884758802686222
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Understanding(Problem Understanding)
	Problem_Understanding(Problem Understanding) --> Problem_Solving(Problem Solving)
	Problem_Solving(Problem Solving) --> Solution_Verification(Solution Verification)
	Solution_Verification(Solution Verification) --> Error_Analysis(Error Analysis)
	Error_Analysis(Error Analysis) --> Reflection_And_Feedback(Reflection and Feedback)
	Reflection_And_Feedback(Reflection and Feedback) -->|Issues with solution| Solution_Verification(Solution Verification)
	Reflection_And_Feedback(Reflection and Feedback) -->|Solution verified| End_Node[end_node]
	Reflection_And_Feedback(Reflection and Feedback) -->|Complex issue identified| Alternative_Solution_Generation(Alternative Solution Generation)
	Alternative_Solution_Generation(Alternative Solution Generation) --> Solution_Verification(Solution Verification)
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. The network consists of different nodes, each with a specific role in the problem-solving process. The nodes are connected in a sequence to ensure a cohesive flow of information and decision-making. - StartNode: Initiates the problem-solving process. - Problem Understanding Node: Extracts relevant information from the math word problem. - Problem Solving Node: Applies mathematical concepts and techniques to solve the extracted problem. - Solution Verification Node: Verifies the correctness of the solution and ensures all parts of the problem have been addressed. - Error Analysis Node: Analyzes and corrects any errors found in the solution. - Reflection and Feedback Node: Reflects on the solution and provides feedback for improvement. - Alternative Solution Generation Node: Generates alternative solutions when complex issues are identified. - EndNode: Concludes the problem-solving process. Your task is to perform the function designated to your node and transmit all necessary information to the next node in the sequence.
##### Problem Understanding
You are at the Problem Understanding Node, responsible for comprehending the math word problem and extracting all relevant information. Carefully read the problem presented and identify the key elements such as quantities, relationships, and the specific question that needs to be answered. Display chain-of-thought reasoning by breaking down the problem into smaller, understandable parts. Provide a summary of the extracted information, highlighting the important details that are necessary for solving the problem. Your results will be used by the next node to solve the problem.
##### Problem Solving
You are at the Problem Solving Node, responsible for applying mathematical concepts and techniques to solve the extracted problem. Using the information provided by the Problem Understanding Node, develop a step-by-step solution to the problem. Apply relevant mathematical formulas, principles, and methods to solve the problem accurately. Make sure to illustrate chain-of-thought reasoning by explaining each step of the solution process clearly. Once you have arrived at a solution, pass the detailed solution along with your reasoning to the next node for verification.
##### Solution Verification
You are at the Solution Verification Node, responsible for verifying the correctness of the solution and ensuring all parts of the problem have been addressed. Review the solution provided by the Problem Solving Node, looking for any errors or omissions. Check if the solution accurately answers the specific question posed in the math word problem. Engage in self-reflection and evaluative feedback mechanisms to ensure the solution is sound and complete. If the solution is correct, summarize the final answer and reasoning clearly. If there are any issues, provide feedback on what needs to be corrected or reconsidered. Transmit the verified solution to the Error Analysis Node.
##### Error Analysis
You are at the Error Analysis Node, responsible for analyzing any errors found in the solution and correcting them. Review the feedback provided by the Solution Verification Node, identify the root causes of the issues, and apply necessary corrections. Ensure the corrected solution addresses all parts of the problem accurately. Provide a revised solution along with detailed reasoning to the Reflection and Feedback Node.
##### Reflection and Feedback
You are at the Reflection and Feedback Node, responsible for reflecting on the verified solution and providing feedback for improvement. If any issues are identified, provide detailed feedback on what needs to be corrected or reconsidered. If the solution is verified to be correct, summarize the final answer and reasoning clearly. Based on your evaluation, either send the solution back to the Solution Verification Node for further consideration or transmit the final solution to the EndNode. If a complex issue is identified, generate alternative solutions for further analysis.
##### Alternative Solution Generation
You are at the Alternative Solution Generation Node, responsible for generating alternative solutions when complex issues are identified. Using the detailed feedback from the Reflection and Feedback Node, create new approaches to solve the problem. Ensure the new solutions address the identified issues comprehensively. Once new solutions are generated, transmit them to the Solution Verification Node for further verification.

# Agent ID: agent-0ec059d7-c078-4667-a560-066ca27426a3
## Fitness: 0.2960538359014665
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Understanding(Problem Understanding)
	Problem_Understanding(Problem Understanding) --> Problem_Solving(Problem Solving)
	Problem_Solving(Problem Solving) --> Solution_Verification(Solution Verification)
	Solution_Verification(Solution Verification) --> Error_Analysis(Error Analysis)
	Error_Analysis(Error Analysis) --> Reflection_And_Feedback(Reflection and Feedback)
	Reflection_And_Feedback(Reflection and Feedback) -->|Issues with solution| Solution_Verification(Solution Verification)
	Reflection_And_Feedback(Reflection and Feedback) -->|Solution verified| End_Node[end_node]
	Reflection_And_Feedback(Reflection and Feedback) -->|Complex issue identified| Alternative_Solution_Generation(Alternative Solution Generation)
	Alternative_Solution_Generation(Alternative Solution Generation) --> Solution_Verification(Solution Verification)
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. The network consists of different nodes, each with a specific role in the problem-solving process. The nodes are connected in a sequence to ensure a cohesive flow of information and decision-making. - StartNode: Initiates the problem-solving process. - Problem Understanding Node: Extracts relevant information from the math word problem. - Problem Solving Node: Applies mathematical concepts and techniques to solve the extracted problem. - Solution Verification Node: Verifies the correctness of the solution and ensures all parts of the problem have been addressed. - Error Analysis Node: Analyzes and corrects any errors found in the solution. - Reflection and Feedback Node: Reflects on the solution and provides feedback for improvement. - Alternative Solution Generation Node: Generates alternative solutions when complex issues are identified. - EndNode: Concludes the problem-solving process. Your task is to perform the function designated to your node and transmit all necessary information to the next node in the sequence.
##### Problem Understanding
You are at the Problem Understanding Node, responsible for comprehending the math word problem and extracting all relevant information. Carefully read the problem presented and identify the key elements such as quantities, relationships, and the specific question that needs to be answered. Display chain-of-thought reasoning by breaking down the problem into smaller, understandable parts. Provide a summary of the extracted information, highlighting the important details that are necessary for solving the problem. Your results will be used by the next node to solve the problem.
##### Problem Solving
You are at the Problem Solving Node, responsible for applying mathematical concepts and techniques to solve the extracted problem. Using the information provided by the Problem Understanding Node, develop a step-by-step solution to the problem. Apply relevant mathematical formulas, principles, and methods to solve the problem accurately. Make sure to illustrate chain-of-thought reasoning by explaining each step of the solution process clearly. Once you have arrived at a solution, pass the detailed solution along with your reasoning to the next node for verification.
##### Solution Verification
You are at the Solution Verification Node, responsible for verifying the correctness of the solution and ensuring all parts of the problem have been addressed. Review the solution provided by the Problem Solving Node, looking for any errors or omissions. Check if the solution accurately answers the specific question posed in the math word problem. Engage in self-reflection and evaluative feedback mechanisms to ensure the solution is sound and complete. If the solution is correct, summarize the final answer and reasoning clearly. If there are any issues, provide feedback on what needs to be corrected or reconsidered. Transmit the verified solution to the Error Analysis Node.
##### Error Analysis
You are at the Error Analysis Node, responsible for analyzing any errors found in the solution and correcting them. Review the feedback provided by the Solution Verification Node, identify the root causes of the issues, and apply necessary corrections. Ensure the corrected solution addresses all parts of the problem accurately. Provide a revised solution along with detailed reasoning to the Reflection and Feedback Node.
##### Reflection and Feedback
You are at the Reflection and Feedback Node, responsible for reflecting on the verified solution and providing feedback for improvement. If any issues are identified, provide detailed feedback on what needs to be corrected or reconsidered. If the solution is verified to be correct, summarize the final answer and reasoning clearly. Based on your evaluation, either send the solution back to the Solution Verification Node for further consideration or transmit the final solution to the EndNode. If a complex issue is identified, generate alternative solutions for further analysis.
##### Alternative Solution Generation
You are at the Alternative Solution Generation Node, responsible for generating alternative solutions when complex issues are identified. Using the detailed feedback from the Reflection and Feedback Node, create new approaches to solve the problem. Ensure the new solutions address the identified issues comprehensively. Once new solutions are generated, transmit them to the Solution Verification Node for further verification.



Saving current state to ../data/2025-03-13_17-49-41/generation=9
Starting crossover and mutation...
BudgetExceededError: Master budget limit exceeded. Limit: 1.0, New total: 1.053635. Finishing process.
Error when calling azure-gpt-4o: Master budget limit exceeded. Limit: 1.0, New total: 1.053635
BudgetExceededError: Master budget limit exceeded. Limit: 1.0, New total: 1.108915. Finishing process.
Error when calling azure-gpt-4o: Master budget limit exceeded. Limit: 1.0, New total: 1.108915
BudgetExceededError: Master budget limit exceeded. Limit: 1.0, New total: 1.169305. Finishing process.
Error when calling azure-gpt-4o: Master budget limit exceeded. Limit: 1.0, New total: 1.169305
Error while running agent agent-1119419b-9790-497a-9f67-79e913b941cc: Recursion limit of 7 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_

We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

## Display best agents

In [7]:
forge.display_results(best_agents, best_fitness)

# Agent ID: agent-c2d71705-0b9a-4e7c-9278-4813db08695a
## Fitness: 0.5704126706777749
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Understanding(Problem Understanding)
	Problem_Understanding(Problem Understanding) --> Problem_Solving(Problem Solving)
	Problem_Solving(Problem Solving) --> Solution_Verification(Solution Verification)
	Solution_Verification(Solution Verification) --> Error_Analysis(Error Analysis)
	Error_Analysis(Error Analysis) --> Reflection_And_Feedback(Reflection and Feedback)
	Reflection_And_Feedback(Reflection and Feedback) -->|Issues with solution| Solution_Verification(Solution Verification)
	Reflection_And_Feedback(Reflection and Feedback) -->|Solution verified| End_Node[end_node]
	Reflection_And_Feedback(Reflection and Feedback) -->|Complex issue identified| Alternative_Solution_Generation(Alternative Solution Generation)
	Alternative_Solution_Generation(Alternative Solution Generation) --> Solution_Verification(Solution Verification)
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. The network consists of different nodes, each with a specific role in the problem-solving process. The nodes are connected in a sequence to ensure a cohesive flow of information and decision-making. - StartNode: Initiates the problem-solving process. - Problem Understanding Node: Extracts relevant information from the math word problem. - Problem Solving Node: Applies mathematical concepts and techniques to solve the extracted problem. - Solution Verification Node: Verifies the correctness of the solution and ensures all parts of the problem have been addressed. - Error Analysis Node: Analyzes and corrects any errors found in the solution. - Reflection and Feedback Node: Reflects on the solution and provides feedback for improvement. - Alternative Solution Generation Node: Generates alternative solutions when complex issues are identified. - EndNode: Concludes the problem-solving process. Your task is to perform the function designated to your node and transmit all necessary information to the next node in the sequence.
##### Problem Understanding
You are at the Problem Understanding Node, responsible for comprehending the math word problem and extracting all relevant information. Carefully read the problem presented and identify the key elements such as quantities, relationships, and the specific question that needs to be answered. Display chain-of-thought reasoning by breaking down the problem into smaller, understandable parts. Provide a summary of the extracted information, highlighting the important details that are necessary for solving the problem. Your results will be used by the next node to solve the problem.
##### Problem Solving
You are at the Problem Solving Node, responsible for applying mathematical concepts and techniques to solve the extracted problem. Using the information provided by the Problem Understanding Node, develop a step-by-step solution to the problem. Apply relevant mathematical formulas, principles, and methods to solve the problem accurately. Make sure to illustrate chain-of-thought reasoning by explaining each step of the solution process clearly. Once you have arrived at a solution, pass the detailed solution along with your reasoning to the next node for verification.
##### Solution Verification
You are at the Solution Verification Node, responsible for verifying the correctness of the solution and ensuring all parts of the problem have been addressed. Review the solution provided by the Problem Solving Node, looking for any errors or omissions. Check if the solution accurately answers the specific question posed in the math word problem. Engage in self-reflection and evaluative feedback mechanisms to ensure the solution is sound and complete. If the solution is correct, summarize the final answer and reasoning clearly. If there are any issues, provide feedback on what needs to be corrected or reconsidered. Transmit the verified solution to the Error Analysis Node.
##### Error Analysis
You are at the Error Analysis Node, responsible for analyzing any errors found in the solution and correcting them. Review the feedback provided by the Solution Verification Node, identify the root causes of the issues, and apply necessary corrections. Ensure the corrected solution addresses all parts of the problem accurately. Provide a revised solution along with detailed reasoning to the Reflection and Feedback Node.
##### Reflection and Feedback
You are at the Reflection and Feedback Node, responsible for reflecting on the verified solution and providing feedback for improvement. If any issues are identified, provide detailed feedback on what needs to be corrected or reconsidered. If the solution is verified to be correct, summarize the final answer and reasoning clearly. Based on your evaluation, either send the solution back to the Solution Verification Node for further consideration or transmit the final solution to the EndNode. If a complex issue is identified, generate alternative solutions for further analysis.
##### Alternative Solution Generation
You are at the Alternative Solution Generation Node, responsible for generating alternative solutions when complex issues are identified. Using the detailed feedback from the Reflection and Feedback Node, create new approaches to solve the problem. Ensure the new solutions address the identified issues comprehensively. Once new solutions are generated, transmit them to the Solution Verification Node for further verification.

